In [7]:
import requests
import json
import numpy
import matplotlib.pyplot as plt
import pandas as pd
from config import bls_key
from config import g_key

In [8]:
# Read unemployment csv into a dataframe
unemployment_file = pd.read_csv("unemployment.csv", header=None, encoding="iso-8859-1")
unemployment_file.head(20)

,0,1,2,3,4,5,6,7,8
0,"Labor force data by county, not seasonally adj...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,State,County,NaN,NaN,NaN,NaN,NaN,Unemploy-
3,NaN,FIPS,FIPS,NaN,NaN,Labor,NaN,NaN,ment Rate
4,LAUS Code,Code,Code,County Name/State Abbreviation,Period,Force,Employed,Unemployed,(%)
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,CN0100100000000,01,001,"Autauga County, AL",Nov-17,"25,871","25,085",786,3.0
7,CN0100300000000,01,003,"Baldwin County, AL",Nov-17,"90,382","87,439","2,943",3.3
8,CN0100500000000,01,005,"Barbour County, AL",Nov-17,"8,148","7,766",382,4.7
9,CN0100700000000,01,007,"Bibb County, AL",Nov-17,"8,455","8,177",278,3.3


In [9]:
# Rename columns
unemployment_df = unemployment_file.rename(columns={0: "County Code", 1: "State FIPS", 2: "County FIPS", 
                                  3: "County_State", 4: "Month", 5: "Labor Force", 6: "Employed", 
                                  7: "Unemployed", 8: "Unemployment Rate (%)"})

unemployment_df.head(10)

,County Code,State FIPS,County FIPS,County_State,Month,Labor Force,Employed,Unemployed,Unemployment Rate (%)
0,"Labor force data by county, not seasonally adj...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,State,County,NaN,NaN,NaN,NaN,NaN,Unemploy-
3,NaN,FIPS,FIPS,NaN,NaN,Labor,NaN,NaN,ment Rate
4,LAUS Code,Code,Code,County Name/State Abbreviation,Period,Force,Employed,Unemployed,(%)
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,CN0100100000000,01,001,"Autauga County, AL",Nov-17,"25,871","25,085",786,3.0
7,CN0100300000000,01,003,"Baldwin County, AL",Nov-17,"90,382","87,439","2,943",3.3
8,CN0100500000000,01,005,"Barbour County, AL",Nov-17,"8,148","7,766",382,4.7
9,CN0100700000000,01,007,"Bibb County, AL",Nov-17,"8,455","8,177",278,3.3


In [10]:
# Remove extra rows that were included from csv formatting
unemployment_df=unemployment_df.drop(unemployment_df.index[:6])
unemployment_df.head(10)

,County Code,State FIPS,County FIPS,County_State,Month,Labor Force,Employed,Unemployed,Unemployment Rate (%)
6,CN0100100000000,01,001,"Autauga County, AL",Nov-17,"25,871","25,085",786,3.0
7,CN0100300000000,01,003,"Baldwin County, AL",Nov-17,"90,382","87,439","2,943",3.3
8,CN0100500000000,01,005,"Barbour County, AL",Nov-17,"8,148","7,766",382,4.7
9,CN0100700000000,01,007,"Bibb County, AL",Nov-17,"8,455","8,177",278,3.3
10,CN0100900000000,01,009,"Blount County, AL",Nov-17,"24,396","23,641",755,3.1
11,CN0101100000000,01,011,"Bullock County, AL",Nov-17,"4,832","4,636",196,4.1
12,CN0101300000000,01,013,"Butler County, AL",Nov-17,"8,842","8,462",380,4.3
13,CN0101500000000,01,015,"Calhoun County, AL",Nov-17,"45,351","43,587","1,764",3.9
14,CN0101700000000,01,017,"Chambers County, AL",Nov-17,"15,210","14,701",509,3.3
15,CN0101900000000,01,019,"Cherokee County, AL",Nov-17,"11,027","10,671",356,3.2


In [11]:
# Use .loc to find unemployment rate at the end of the year
unemployment_df = unemployment_df.loc[unemployment_df["Month"] == "Dec-17"]

# Reset index values
unemployment_df = unemployment_df.reset_index(drop=True)

unemployment_df.head(10)

,County Code,State FIPS,County FIPS,County_State,Month,Labor Force,Employed,Unemployed,Unemployment Rate (%)
0,CN0100100000000,01,001,"Autauga County, AL",Dec-17,"25,733","24,952",781,3.0
1,CN0100300000000,01,003,"Baldwin County, AL",Dec-17,"90,001","87,006","2,995",3.3
2,CN0100500000000,01,005,"Barbour County, AL",Dec-17,"8,074","7,704",370,4.6
3,CN0100700000000,01,007,"Bibb County, AL",Dec-17,"8,412","8,143",269,3.2
4,CN0100900000000,01,009,"Blount County, AL",Dec-17,"24,164","23,434",730,3.0
5,CN0101100000000,01,011,"Bullock County, AL",Dec-17,"4,689","4,480",209,4.5
6,CN0101300000000,01,013,"Butler County, AL",Dec-17,"8,753","8,374",379,4.3
7,CN0101500000000,01,015,"Calhoun County, AL",Dec-17,"45,160","43,425","1,735",3.8
8,CN0101700000000,01,017,"Chambers County, AL",Dec-17,"15,187","14,693",494,3.3
9,CN0101900000000,01,019,"Cherokee County, AL",Dec-17,"10,922","10,579",343,3.1


In [24]:
# All columns are strings with hidden white space. Loop through rows of numerical columns to strip white space

string = unemployment_df["Labor Force"][0]
print(string.strip())

for x in range(len(unemployment_df)):
    unemployment_df["Labor Force"][x].strip()
    unemployment_df["Employed"][x].strip()
    unemployment_df["Unemployed"][x].strip()
    unemployment_df["Unemployment Rate (%)"][x].strip()
    
print(string)    

25,733
25,733     


In [39]:
base_url = "https://api.bls.gov/publicAPI/v2/timeseries/data/"
prefix = "LAUCN"


response = requests.get(f"{base_url}/LAUST110000000000003?registrationkey=de81d9b3e1824aef9968747fec0e3999")

In [40]:
response.json()

{'status': 'REQUEST_SUCCEEDED',
 'responseTime': 167,
 'message': [],
 'Results': {'series': [{'seriesID': 'LAUST110000000000003',
    'data': [{'year': '2018',
      'period': 'M12',
      'periodName': 'December',
      'value': '4.9',
      'footnotes': [{'code': 'D',
        'text': 'Reflects revised population controls and model reestimation.'}]},
     {'year': '2018',
      'period': 'M11',
      'periodName': 'November',
      'value': '5.1',
      'footnotes': [{'code': 'D',
        'text': 'Reflects revised population controls and model reestimation.'}]},
     {'year': '2018',
      'period': 'M10',
      'periodName': 'October',
      'value': '5.3',
      'footnotes': [{'code': 'D',
        'text': 'Reflects revised population controls and model reestimation.'}]},
     {'year': '2018',
      'period': 'M09',
      'periodName': 'September',
      'value': '5.5',
      'footnotes': [{'code': 'D',
        'text': 'Reflects revised population controls and model reestimation.'}]

In [37]:
import geocoder

results = geocoder.google("102 SW 6th Ave, Miami, FL 33130", key=g_key)
print(results.postal)

33130
